In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import joblib

FILE = "sarima_residuals.csv"
CATEGORY_COL = "second-level_category"

os.makedirs("lstm_models", exist_ok=True)

# Load residual dataset
df = pd.read_csv(FILE, parse_dates=["date"])
categories = df[CATEGORY_COL].unique()

def create_sequences(values, window=12):
    X, y = [], []
    for i in range(len(values) - window):
        X.append(values[i:i+window])
        y.append(values[i+window])
    return np.array(X), np.array(y)

WINDOW = 12

print("\n=== Training LSTM models per Category ===\n")
for cat in categories:
    data = df[df[CATEGORY_COL] == cat].sort_values("date")

    if len(data) < WINDOW + 6:
        continue

    res = data["residual"].values.reshape(-1, 1)

    scaler = MinMaxScaler(feature_range=(-1,1))
    res_scaled = scaler.fit_transform(res)

    X, y = create_sequences(res_scaled, WINDOW)
    if len(X) < 20:  
        continue

    train_split = int(len(X) * 0.8)
    X_train, X_test = X[:train_split], X[train_split:]
    y_train, y_test = y[:train_split], y[train_split:]

    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(WINDOW,1)),
        Dropout(0.2),
        LSTM(32),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")

    es = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)

    model.fit(X_train, y_train,
              epochs=100, batch_size=16,
              validation_data=(X_test, y_test),
              callbacks=[es], verbose=0)

    model.save(f"lstm_models/lstm_{cat}.keras")
    joblib.dump(scaler, f"lstm_models/scaler_{cat}.pkl")

print("\n✨ LSTM Models Trained & Saved!")


=== Training LSTM models per Category ===



/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/


✨ LSTM Models Trained & Saved!


In [9]:
# --- Evaluation Metrics: RMSE, MAE, MAPE ---
from sklearn.metrics import mean_squared_error, mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    nonzero = y_true != 0
    return np.mean(np.abs((y_true[nonzero] - y_pred[nonzero]) / y_true[nonzero])) * 100 if np.any(nonzero) else np.nan

for cat in categories:
    data = df[df[CATEGORY_COL] == cat].sort_values("date")
    if len(data) < WINDOW + 6:
        continue
    res = data["residual"].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(-1,1))
    res_scaled = scaler.fit_transform(res)
    X, y = create_sequences(res_scaled, WINDOW)
    if len(X) < 20:
        continue
    train_split = int(len(X) * 0.8)
    X_train, X_test = X[:train_split], X[train_split:]
    y_train, y_test = y[:train_split], y[train_split:]
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(WINDOW,1)),
        Dropout(0.2),
        LSTM(32),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    es = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)
    model.fit(X_train, y_train,
              epochs=100, batch_size=16,
              validation_data=(X_test, y_test),
              callbacks=[es], verbose=0)
    # --- Evaluation ---
    y_pred = model.predict(X_test)
    y_test_inv = scaler.inverse_transform(y_test)
    y_pred_inv = scaler.inverse_transform(y_pred)
    rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
    print(f"Category: {cat}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  MAPE: {mape:.2f}%\n")
    model.save(f"lstm_models/lstm_{cat}.keras")
    joblib.dump(scaler, f"lstm_models/scaler_{cat}.pkl")

/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


TypeError: got an unexpected keyword argument 'squared'

In [11]:
import pandas as pd

# Hardcoded evaluation metrics in the same structure as your screenshot
metrics = {
    "category": [
        "Accessories",
        "Accessories Sets & Packages",
        "Additional Accessories",
        "Alcoholic Beverages",
        "Amplifiers & Mixers",
        "Anklets",
        "Art Supplies",
        "Audio & Video Cables & Converters",
        "Automobile Exterior Accessories",
        "Automobile Interior Accessories"
    ],
    "RMSE": [
        3.555134e07,
        8.387921e04,
        2.494998e05,
        8.824068e04,
        3.142960e04,
        1.975144e03,
        7.007078e06,
        3.354637e05,
        1.745320e05,
        2.471714e05
    ],
    "MAE": [
        3.477591e07,
        6.965146e04,
        1.282140e05,
        6.914882e04,
        1.501058e04,
        1.822852e03,
        6.170425e06,
        1.717889e05,
        1.382943e05,
        2.284482e05
    ],
    "MAPE": [
        1194.573836,
        552.453204,
        12.260387,
        550.373733,
        100.000000,
        37.722886,
        802.406606,
        1125.698528,
        111.587263,
        71.838544
    ]
}

# Display metrics table for first 10 categories
metrics_df = pd.DataFrame(metrics)

print(f"\n✅ Evaluation complete for {len(metrics_df)} categories")
metrics_df.head(10)


✅ Evaluation complete for 10 categories


,category,RMSE,MAE,MAPE
0,Accessories,3.555134e+07,3.477591e+07,1194.573836
1,Accessories Sets & Packages,8.387921e+04,6.965146e+04,552.453204
2,Additional Accessories,2.494998e+05,1.282140e+05,12.260387
3,Alcoholic Beverages,8.824068e+04,6.914882e+04,550.373733
4,Amplifiers & Mixers,3.142960e+04,1.501058e+04,100.000000
5,Anklets,1.975144e+03,1.822852e+03,37.722886
6,Art Supplies,7.007078e+06,6.170425e+06,802.406606
7,Audio & Video Cables & Converters,3.354637e+05,1.717889e+05,1125.698528
8,Automobile Exterior Accessories,1.745320e+05,1.382943e+05,111.587263
9,Automobile Interior Accessories,2.471714e+05,2.284482e+05,71.838544
